In [1]:
import os
import json
import pandas as pd

In [2]:
uri = "../temp/matches.json"

In [3]:
os.path.exists(uri)

True

In [4]:
with open(uri) as file:
    data = json.load(file)

In [5]:
# keys = data['Premier League'][0].keys()
# keys

In [6]:
pl_data = data['Premier League'][0]

In [7]:
# To get data of second matchday
# data['Premier League'][1]

In [8]:
season = int(pl_data['filters']['season'])
season

2023

In [9]:
matchday = int(pl_data['filters']['matchday'])
matchday

1

In [10]:
# pl_data['resultSet']

##### FIRST_MATCHDAY = pl_data['resultSet']['first']
##### LAST_MATCHDAY = pl_data['resultSet']['last']

##### FIRST_MATCHDAY, LAST_MATCHDAY

In [11]:
LEAGUE_ID = pl_data['competition']['id']

LEAGUE_ID

2021

In [12]:
# pl_data['matches']

In [13]:
# pl_data['matches'][0]

In [14]:
# pl_data['matches'][0].keys()

In [15]:
FIRST_MATCHDAY = pl_data['matches'][0]['season']['startDate']
LAST_MATCHDAY = pl_data['matches'][0]['season']['endDate']
FIRST_MATCHDAY, LAST_MATCHDAY

('2023-08-11', '2024-05-19')

In [16]:
match_id = pl_data['matches'][0]['id']

match_id

435943

In [17]:
matchdate = pl_data['matches'][0]['utcDate']
matchdate

'2023-08-11T19:00:00Z'

In [18]:
home_team = pl_data['matches'][0]['homeTeam']['shortName']
home_team

'Burnley'

In [19]:
away_team = pl_data['matches'][0]['awayTeam']['shortName']
away_team

'Man City'

In [20]:
home_score = pl_data['matches'][0]['score']['fullTime']['home']
home_score

0

In [21]:
away_score = pl_data['matches'][0]['score']['fullTime']['away']
away_score

3

In [98]:
def get_match_points(home_score: int, away_score: int) -> tuple[int, int]:

    if isinstance(home_score, int):
        if home_score == away_score:
            home_team_points, away_team_points = 1, 1
        elif home_score > away_score:
            home_team_points = 3
            away_team_points = 0
        else:
            home_team_points = 0
            away_team_points = 3
    else:
        home_team_points = None
        away_team_points = None
    
    return home_team_points, away_team_points 

In [99]:
def get_goal_difference(home_score: int, away_score: int) -> tuple[int, int]:
    if isinstance(home_score, int):
        home_gd = home_score - away_score
        away_gd = -home_gd
    else:
        home_gd = None
        away_gd = None

    return home_gd, -home_gd

In [100]:
leagues = {'Premier League': [38, 10], 
          'Bundesliga': [34, 9], 
          'Ligue 1': [34, 9], 
          'La Liga': [38, 10], 
          'Serie A': [38, 10]}

In [128]:
columns = ['league', 'season', 'league_id', 'season_start_date', 'season_end_date', 
           'matchday', 'match_id', 'match_date', 'home_team', 'away_team', 
           'home_score', 'away_score', 'home_team_points_earned', 'away_team_points_earned', 
           'home_team_gd', 'away_team_gd', 'match_referee']

df = pd.DataFrame(columns = columns)

for league, total_matchdays in leagues.items():

    print(f"Getting data for {league}...")
    # Get Season, League_id, first and last matchday dates only once as that is a constant
    firstday_data = data[league][0]
    SEASON = int(firstday_data['filters']['season'])
    LEAGUE_ID = firstday_data['competition']['id']
    FIRST_MATCHDAY = firstday_data['matches'][0]['season']['startDate']
    LAST_MATCHDAY = firstday_data['matches'][0]['season']['endDate']
    
    # Also need to consider status
    for mday in range(0, total_matchdays[0]):
        # print(f"\tGetting matchday {mday+1} data...")
        pl_data = data[league][mday]

        # No data available for matchday 23 of Serie A :(
        if league == 'Serie A' and mday == 22:
            pass
        else:
            matchday = int(pl_data['filters']['matchday'])
        
            for match_num in range(0, total_matchdays[1]):
                # print(f"\t\tGetting data for match {match_num + 1}")
                match_data = pl_data['matches'][match_num]
                match_id = match_data['id']
                matchdate = match_data['utcDate']
                home_team = match_data['homeTeam']['shortName']
                away_team = match_data['awayTeam']['shortName']
                home_score = match_data['score']['fullTime']['home']
                away_score = match_data['score']['fullTime']['away']
                      
                try:
                    match_referee = match_data['referees'][0]['name']
                except IndexError:
                    match_referee = ''
                
                home_team_points, away_team_points = get_match_points(home_score=home_score, away_score=away_score)
                
                try:
                    home_team_gd, away_team_gd = get_goal_difference(home_score=home_score, away_score=away_score)
                except TypeError:
                    home_team_gd, away_team_gd = None, None
                    
                df.loc[len(df.index)] = [league, SEASON, LEAGUE_ID, FIRST_MATCHDAY, LAST_MATCHDAY, 
                                         matchday, match_id, matchdate, home_team, away_team, 
                                         home_score, away_score, home_team_points, away_team_points, 
                                         home_team_gd, away_team_gd, match_referee]

print("Successfully completed adding data for top 5 leagues...")

Getting data for Premier League...
Getting data for Bundesliga...
Getting data for Ligue 1...
Getting data for La Liga...
Getting data for Serie A...
Successfully completed adding data for top 5 leagues...


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1742 entries, 0 to 1741
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   league                   1742 non-null   object
 1   season                   1742 non-null   int64 
 2   league_id                1742 non-null   int64 
 3   season_start_date        1742 non-null   object
 4   season_end_date          1742 non-null   object
 5   matchday                 1742 non-null   int64 
 6   match_id                 1742 non-null   int64 
 7   match_date               1742 non-null   object
 8   home_team                1742 non-null   object
 9   away_team                1742 non-null   object
 10  home_score               1292 non-null   object
 11  away_score               1292 non-null   object
 12  home_team_points_earned  1292 non-null   object
 13  away_team_points_earned  1292 non-null   object
 14  home_team_gd             1292 non-null   obje

In [130]:
# Convert some columns to correct dtype
df.season_start_date = pd.to_datetime(df.season_start_date)
df.season_end_date = pd.to_datetime(df.season_end_date)
df.match_date = pd.to_datetime(df.match_date).dt.date
df.match_date = pd.to_datetime(df.match_date)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1742 entries, 0 to 1741
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   league                   1742 non-null   object        
 1   season                   1742 non-null   int64         
 2   league_id                1742 non-null   int64         
 3   season_start_date        1742 non-null   datetime64[ns]
 4   season_end_date          1742 non-null   datetime64[ns]
 5   matchday                 1742 non-null   int64         
 6   match_id                 1742 non-null   int64         
 7   match_date               1742 non-null   datetime64[ns]
 8   home_team                1742 non-null   object        
 9   away_team                1742 non-null   object        
 10  home_score               1292 non-null   object        
 11  away_score               1292 non-null   object        
 12  home_team_points_earned  1292 non-null 

In [132]:
df.isnull().sum()

league                       0
season                       0
league_id                    0
season_start_date            0
season_end_date              0
matchday                     0
match_id                     0
match_date                   0
home_team                    0
away_team                    0
home_score                 450
away_score                 450
home_team_points_earned    450
away_team_points_earned    450
home_team_gd               450
away_team_gd               450
match_referee                0
dtype: int64

In [133]:
df = df.fillna('Match not completed')

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1742 entries, 0 to 1741
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   league                   1742 non-null   object        
 1   season                   1742 non-null   int64         
 2   league_id                1742 non-null   int64         
 3   season_start_date        1742 non-null   datetime64[ns]
 4   season_end_date          1742 non-null   datetime64[ns]
 5   matchday                 1742 non-null   int64         
 6   match_id                 1742 non-null   int64         
 7   match_date               1742 non-null   datetime64[ns]
 8   home_team                1742 non-null   object        
 9   away_team                1742 non-null   object        
 10  home_score               1742 non-null   object        
 11  away_score               1742 non-null   object        
 12  home_team_points_earned  1742 non-null 

In [135]:
df[df['home_score'] != 'Match not completed']['home_score'].value_counts()

home_score
1    426
2    312
0    275
3    180
4     63
5     29
6      3
7      2
8      2
Name: count, dtype: int64

In [136]:
df[df['away_score'] != 'Match not completed']['away_score'].value_counts()

away_score
1    435
0    393
2    277
3    133
4     39
5      9
6      4
8      1
7      1
Name: count, dtype: int64

In [158]:
df[(df['away_score'] == 7) | (df['away_score'] == 8)]

,league,season,league_id,season_start_date,season_end_date,matchday,match_id,match_date,home_team,away_team,home_score,away_score,home_team_points_earned,away_team_points_earned,home_team_gd,away_team_gd,match_referee
59,Premier League,2023,2021,2023-08-11,2024-05-19,6,436002,2023-09-24,Sheffield Utd,Newcastle,0,8,0,3,-8,8,Stuart Attwell
1021,La Liga,2023,2014,2023-08-13,2024-05-26,3,438502,2023-08-28,Rayo Vallecano,Atleti,0,7,0,3,-7,7,José Munuera Montero
